In [1]:
import torch
import numpy as np
from torch import Tensor
from dataclasses import dataclass
from pathlib import Path
from rdkit.Chem import GetPeriodicTable
torch.set_default_dtype(torch.float64)

ptable = GetPeriodicTable()

In [2]:
amutokg = 1.660539040e-27
kgtoamu = 1.0/amutokg
metokg  = 9.10938356e-31
kgtome  = 1.0/metokg
amutoau = amutokg*kgtome # AMU to a atomic units
fstoau = 41.3413733365614
autoev = 27.21138505
evtoau = 1.0/autoev

atomic_mass_nist = [
1.00794075,  4.00260193,  6.94003660,  9.01218307,
10.81102805, 12.01073590, 14.00670321, 15.99940492,
18.99840316, 20.18004638, 22.98976928, 24.30505162,
26.98153853, 28.08549871, 30.97376200, 32.06478741,
35.45293758, 39.94779856, 39.09830091, 40.07802251,
44.95590828, 47.86674496, 50.94146504, 51.99613176,
54.93804391, 55.84514443, 58.93319429, 58.69334711,
63.54603995, 65.37778253, 69.72306607, 72.62755016,
74.92159457, 78.95938856, 79.90352778, 83.79800000,
85.46766360, 87.61664447, 88.90584030, 91.22364160,
92.90637300, 95.95978854, 97.90721240, 101.06494014,
]
symbols = [ptable.GetElementSymbol(e+1) for e in range(len(atomic_mass_nist))]
atomic_mass_nist = dict(zip(symbols, atomic_mass_nist))
atomic_mass_nist

{'H': 1.00794075,
 'He': 4.00260193,
 'Li': 6.9400366,
 'Be': 9.01218307,
 'B': 10.81102805,
 'C': 12.0107359,
 'N': 14.00670321,
 'O': 15.99940492,
 'F': 18.99840316,
 'Ne': 20.18004638,
 'Na': 22.98976928,
 'Mg': 24.30505162,
 'Al': 26.98153853,
 'Si': 28.08549871,
 'P': 30.973762,
 'S': 32.06478741,
 'Cl': 35.45293758,
 'Ar': 39.94779856,
 'K': 39.09830091,
 'Ca': 40.07802251,
 'Sc': 44.95590828,
 'Ti': 47.86674496,
 'V': 50.94146504,
 'Cr': 51.99613176,
 'Mn': 54.93804391,
 'Fe': 55.84514443,
 'Co': 58.93319429,
 'Ni': 58.69334711,
 'Cu': 63.54603995,
 'Zn': 65.37778253,
 'Ga': 69.72306607,
 'Ge': 72.62755016,
 'As': 74.92159457,
 'Se': 78.95938856,
 'Br': 79.90352778,
 'Kr': 83.798,
 'Rb': 85.4676636,
 'Sr': 87.61664447,
 'Y': 88.9058403,
 'Zr': 91.2236416,
 'Nb': 92.906373,
 'Mo': 95.95978854,
 'Tc': 97.9072124,
 'Ru': 101.06494014}

In [3]:
mchrg_prod = 3 # 3 Means CID
prog = 8 # 8 means GFN2-xTB
nuc = 14



In [4]:

#   ! MS method
#   method = 0
#   ! logical for too small fragments
#   small=.false.
#   littlemass=.false.
#   ! no checking of input etc
#   check=.false.
#   ! more infos
#   verbose = .false.
#   ! no production
#   prod   =.false.
#   ! no equilibration
#   noeq   =.true.
#   ! test calls
#   eonly  =.false.
#   eonly0 =.false.
#   eonly1 =.false.
#   ! uniform velocity scaling
#   unity=.false.
#   ! check if cid was OK
#   stopcid = .false.
#   starting_md=.false.
#   No_ESI = .false.
#   ! HS-UHF ini only for frag runs
#   iniok  =.true.
#   ! dump every dumpstep MD steps for MOLDEN movie (=4 fs as default)
#   dumpstep=4
#   ! counts the number of QC calls
#   calls=0
#   ! set scaling temp to 0
#   tscale = 0.0_wp

#   ! GBSA Solvation Model
#   !solvent='none'
#   !gsolvstate=0

#   new_velo = 0.0d0
#   ! total MD time including ion tracks
#   ttime=0
#   dtime = 0.0d0
#   ! undefined spin state
#   mspin=0
#   ! neutral M
#   mchrg=0
#   ! is fragmented?
#   fragstate=0

#   t1 = 0.0_wp
#   t2 = 0.0_wp
#   w1 = 0.0_wp
#   w2 = 0.0_wp

# MS Method
method = 0
# If too small, no fragmentation can happen
small = False
littlemass = False

# Skip checking input
check = False

verbose = False


prod = False # Do production runs?
noeq = True # Do equilibration
eonly = False # test calls?
eonly0 = False
eonly1 = False

unity = False # uniform velocity scaling
stopcid = False # check if cid was OK
starting_md = False
No_ESI = False # Do electrospray ionization
iniok = True # HS-UHF initialization for fragmentation runs only
dumpstep = 4 # Dump molden data steps
calls = 0 # Count number of QC calls
tscale = 0.0 # Set temperature scale

new_velo = 0.0
ttime = 0 # Total MD time including ion tracks
dtime = 0.0 # Time step size

mspin = 0 # Molecule spin?
mchrg = 0 # Molecule charge
fragstate = 0 # Is fragmented

t1 = 0.0 # Timing variables (start, end)
t2 = 0.0
w1 = 0.0
w2 = 0.0

In [5]:
from dataclasses import dataclass, field
import torch

@dataclass
class RunSettings:
    tadd: float
    eimp: float
    xyzr: torch.Tensor = field(default_factory=lambda: torch.empty(0, 0, 0))
    velor: torch.Tensor = field(default_factory=lambda: torch.empty(0, 0, 0))
    eimpr: torch.Tensor = field(default_factory=lambda: torch.empty(0))
    taddr: torch.Tensor = field(default_factory=lambda: torch.empty(0))
    velofr: torch.Tensor = field(default_factory=lambda: torch.empty(0, 0))

@dataclass
class CollisionType:
    set_coll: int
    max_coll: int
    
    from dataclasses import dataclass, field

@dataclass
class StructureType:
    nat: int = 0
    nid: int = 0
    nbd: int = 0
    id: torch.Tensor = field(default_factory=lambda: torch.empty(0, dtype=torch.int32))
    num: torch.Tensor = field(default_factory=lambda: torch.empty(0, dtype=torch.int32))
    sym: list = field(default_factory=list)
    xyz: torch.Tensor = field(default_factory=lambda: torch.empty((0, 0), dtype=torch.float32))
    uhf: int = 0
    charge: float = 0.0
    lattice: torch.Tensor = field(default_factory=lambda: torch.empty((0, 0), dtype=torch.float32))
    periodic: torch.Tensor = field(default_factory=lambda: torch.empty(0, dtype=torch.bool))
    bond: torch.Tensor = field(default_factory=lambda: torch.empty((0, 0), dtype=torch.int32))
    comment: str = ""
    info: dict = field(default_factory=dict)
    sdf: list = field(default_factory=list)
    pdb: list = field(default_factory=list)

In [6]:
collauto = False
fullauto = False
manual = True
temprun = False

In [7]:
@dataclass
class Molecule:
    nat: int
    xyz: Tensor
    elem: list[str]
    iat: list[int]
    mass: Tensor
        
    @staticmethod
    def from_xyz(path: Path):
        nat, *arr = Path(path).read_text().split()
        nat = int(nat)
        arr = np.array(arr).reshape(-1,4)
        elem, xyz = arr[:,0], arr[:,1:]
        xyz = xyz.astype(float)
        iat = torch.tensor([ptable.GetAtomicNumber(symbol) for symbol in elem])
        mass = torch.tensor([atomic_mass_nist[symbol] for symbol in elem])
        
        assert nat == len(xyz), f'nat == {nat} while len(xyz) == {len(xyz)}'
        return Molecule(
            nat = nat,
            xyz=xyz,
            elem=elem.tolist(),
            iat=iat,
            mass=mass,
        )
mol_file = '../runs/current/TMPQCXMS/TMP.1/start.xyz'
coord_file = '../runs/current/TMPQCXMS/TMP.1/'
# mol = 
# !cat {mol_file}
mol = Molecule.from_xyz(mol_file)

In [8]:
# This means we have transition metals or s4/5s elems
ECP = False

In [9]:
nuc = mol.nat
ntraj = nuc * 25
ntraj

350

In [10]:
ax: float = 0 # 

def setetemp(nfrag: int, eimp:float) -> float:
    """
    Estimate the electronic temp in Fermi smearing for given IEE and ax
    """
    etemp =  5000. + 20000. * ax
    if eimp > 0 and nfrag <= 1:
        tmp = max(eimp, 0)
        etemp = etemp + tmp * ieetemp
        
    return etemp


ieetemp: float = 0 # TODO: Decode 'relate IEE to eTemp in SCF'
betemp: float = 0 # 
betemp = setetemp(nfrag=1, eimp=-1.0)

In [11]:
iseed: int = 0
torch.manual_seed(iseed)
randx = torch.rand(1,)
randx

tensor([0.9701])

In [12]:
nmax0 = ntraj * 50
nmax0

17500

In [13]:
betemp = 5000.0 # TODO: Any effects from this? "Hardcoded for xTB for now"

In [14]:
tmax = 5 # Max running time in picoseconds
tmax = tmax * 1000.0 # Max running time in femtoseconds
tstep = 0.5 # Timestep in femtoseconds
nmax = tmax / tstep # Total steps
tmax, nmax, tstep

(5000.0, 10000.0, 0.5)

In [15]:
# Convert femtoseconds to atomic units (aus)


eimp0 = 70 # Electronvolts
eimp0 = eimp0 * evtoau # energy in atomic units
tstep, eimp0

(0.5, 2.572452665359642)

In [16]:
# Are there any 4s-4d/5s-5d elements?
noecp = False
ECP = False

In [17]:
mol.iat

tensor([8, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [18]:
nuc

14

In [19]:
mass = mol.mass * amutoau
mass

tensor([29165.1310, 21894.2322, 21894.2322, 21894.2322, 21894.2322,  1837.3636,
         1837.3636,  1837.3636,  1837.3636,  1837.3636,  1837.3636,  1837.3636,
         1837.3636,  1837.3636])

In [20]:
method = 3 # 3 means CID
# TODO 
assert prog != 0, "CID is not suitable with DFTB, why?"

In [21]:
mass

tensor([29165.1310, 21894.2322, 21894.2322, 21894.2322, 21894.2322,  1837.3636,
         1837.3636,  1837.3636,  1837.3636,  1837.3636,  1837.3636,  1837.3636,
         1837.3636,  1837.3636])

In [22]:
velof = 1.0

# Set charge
mchrg_prod = 1
mchrg = mchrg_prod
chrgcont = mchrg

collisions = 0
edum = 0

# cidcheck():
@dataclass
class CollisionType:
    max_coll: int = 6
    set_coll: int = 10
    
coll = CollisionType()

In [23]:
tadd = 0.0
eimp = 0.0
velof = 1.0
# Smaller than 7 atoms doesn't work
# 
small = nuc <= 7
assert not small

In [24]:
starting_md = False

In [25]:
manual = True

In [26]:
# when collsec > 0, fragment until you have collsec fragments
# specifically. First fragmentation can yield max collsec[0] fragments
# second, max collsec[1], etc
collsec = torch.zeros(3)
# when collno > 0, force this many collisions
collno = torch.zeros(3)


ELAB = 40 # eV
maxcoll = 6  # Do this many collisions max

if coll.max_coll != 0:
    print('!!! M+ collisions are user set !!!')
    collisions = coll.max_coll
    new_counter = 1 # This counts total fragments at current step

!!! M+ collisions are user set !!!


### Start Loop for CID collision and subsequent MD simulation

In [27]:
itrj: int = 0
icoll: int = 0
isec: int = 0

In [28]:
isec = 1
icoll = icoll + 1
fragstate = 0

In [29]:
itraj = 1
collisions = 6
mchrg = 1
chrgcont = 1.0

In [31]:
# direc(3),cm(3),cm1(3),cm2(3)
direc, cm, cm1, cm2 = torch.zeros((4, 3), dtype=torch.float32)

if icoll != 1:
    direc = cm2 - cm1
    direc /= torch.norm(direc, p=2)

In [32]:
direc

tensor([0., 0., 0.], dtype=torch.float32)